In [ ]:
import requests,openpyxl;
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup

url ="https://www.acemicromatic.net/product_cat/milling/"
req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
page_soup = soup(webpage, "html.parser")
#text1=page_soup.find("tbody", class_="default-hide click-head show_row_no show_row").find_all("tr", class_="hide_row hide_2")
#print(page_soup.prettify())
excel= openpyxl.Workbook()
sheet=excel.active
sheet.title="Product Info"
sheet.append(["Param_1","Param_2","Model_name","Travels info"])


def extract_travel_info(model_info_url):
    req = Request(model_info_url , headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    page_soup = soup(webpage, "html.parser")
    text3=page_soup.find("div",class_="col-lg-12").find_all("a")
    if len(text3)>=3:
        param1=text3[1].text
        print(param1)
        header=text3[2].text
        print(header)
        travel_section = page_soup.find("tbody", class_="default-hide click-head show_row_no show_row")
        if travel_section:
            for index, td in enumerate(travel_section.find_all("tr", class_="hide_row hide_2")):
                if index == 0:
                    tags = td.find_all("td")[2]
                    content = tags.text.strip()
                    print(content)
                    text2=page_soup.find_all("div", class_="col-md-6 mb-4")
                    if text2:
                       for i in text2:
                          modeltag=i.find("h3")
                          if modeltag!=None:
                              for i in modeltag:
                                   model=i.text.strip()
                                   print(model)
                                   sheet.append([param1,header,model,content])
        else:
            return 'Travel information not found'
    else:
        return "Travel Info not available" 


#model="https://www.acemicromatic.net/aceproduct/spark-xl/"
#print(extract_travel_info(model))

def product_models(link):
    req = Request(link , headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    page_soup = soup(webpage, "html.parser")
    models=page_soup.find_all("div", class_="col-md-4 mb-4 mb-md-0")
    models1=page_soup.find_all("div", class_="col-md-4 mb-4 mb-md-0 only_sub_shw")
    for i in models1:
        productinfolink1=i.find("div", class_="button-wrapper")
        if productinfolink1:
            verticalmachinelink=productinfolink1.find("a")
            machinelink=verticalmachinelink.get("href")
            #print(machinelink)
            machinetravelinfo=remaining_models(machinelink)
            print(machinetravelinfo)
    for i in models:
        productinfolink=i.find("div",class_="button-wrapper")
        if productinfolink:
            infolink1=productinfolink.find("a")
            infolink=infolink1.get("href")
            #print(infolink)
            travelinfo=extract_travel_info(infolink)
            print(travelinfo)
    
def remaining_models(url1):
    req = Request(url1 , headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    page_soup = soup(webpage, "html.parser")
    models2=page_soup.find_all("div", class_="col-md-4 mb-4 mb-md-0")
    for i in models2:
        remainingproduct=i.find("div",class_="button-wrapper")
        if remainingproduct:
            remainingproductlink=remainingproduct.find("a")
            remainingproductlink1=remainingproductlink.get("href")
            #print(remainingproductlink1)
            travelinfo=extract_travel_info(remainingproductlink1)
            print(travelinfo)

products=page_soup.find_all("div", class_="col-md-4 mb-4 mb-md-0 only_sub_shw")
for i in products:
    model_link=i.find("div",class_="button-wrapper")
    if model_link:
        link1=model_link.find("a")
        link=link1.get("href")
        #print(link)
        travelling=product_models(link)
        print(travelling)

excel.save("WebScraping.xlsx")

In [1]:
import pandas as pd
df=pd.read_excel("WebScraping.xlsx");
df.head()
df[["X_travel","Y_travel","Z_travel"]] = df['Travels info'].str.split('/', expand=True)
df.drop(columns=['Travels info'], inplace=True)
import openpyxl

df1="WebScraping.xlsx"
book = openpyxl.load_workbook(df1)
writer = pd.ExcelWriter(df1, engine='openpyxl') 
writer.book = book
sheet_name = 'Sheet1'
df.to_excel(writer, sheet_name=sheet_name, index=False)
writer.save()
writer.close()
